In [ ]:
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv("../data/processed/bf_efficiency_clean.csv")


In [ ]:
X = df[["blood_input_ml", "temperature_c", "humidity_pct", "colony_strength "]]
y = df["egg_output_total"]

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
model.summary()


In [ ]:
df.columns.tolist()